## Random Forest Regression
In the previous section we considered random forests within the context of classification. Random forests can also be made to work in the case of regression (that is, with a continuous rather than a categorical label). The estimator to use for this is the **RandomForestRegressor**, and the syntax is very similar to what we saw earlier. Under the hood a process known as discretization is used to convert a continuous attribute to a categorical one.
### Case Blue-bike
In every major Belgian railway station you can rent a blue-bike. Through a website, a mobile app and an API you can see the real time availability of blue-bikes in the railway station of your choice. The data for the station of Gent Sint-Pieters has been collected every 30 minutes from January till October 2018. The file **`bluebike.csv`** contains a set of observations of the available blue-bikes in the station of Gent-Sint-Pieters. 

The file **`weatherhistory.csv`** contains hourly weather observations for approximately the same period collected by a weather station near Gent.  
  
Since we want to make a prediction per half hour we keep for each observation only “the hour”, for instance for 10.30 pm the field column contains the value 22.5. Weekday coding is as follows: 0 = Monday, 1 = Tuesday, etc. As you can see the “weather” attribute is already hot encoded.  

In [21]:
import pandas as pd
bb = pd.read_csv('data/bluebike.csv',sep=';')

In [22]:
# restrict to all observations as of 2018-01-24
bb = bb[bb['Timestamp'] >= '2018-01-24']
print(bb.head(10))

                Timestamp  SPCapacityTotal  SPCapacityInUse  \
3671  2018-01-24 22:45:41               64               17   
3672  2018-01-24 23:00:02               64               17   
3673  2018-01-24 23:30:02               64               16   
3674  2018-01-25 00:00:02               64               16   
3675  2018-01-25 00:30:05               64               16   
3676  2018-01-25 01:00:06               64               16   
3677  2018-01-25 01:30:10               64               16   
3678  2018-01-25 02:00:03               64               16   
3679  2018-01-25 02:30:08               64               16   
3680  2018-01-25 03:00:07               64               16   

      SPCapacityAvailable  SPCapacityInMaintenance  SPCheckSum  \
3671                   43                        2           2   
3672                   43                        2           2   
3673                   43                        3           2   
3674                   43                 

In [23]:
# drop columns irrelevant for model
bb = bb.drop('SPCapacityTotal',axis=1).drop('SPCapacityInUse',axis=1).drop('SPCapacityInMaintenance',axis=1)\
.drop('SPCheckSum',axis=1).drop('DPCapacityTotal',axis=1).drop('DPCapacityInUse',axis=1).drop('DPCapacityAvailable',axis=1)\
.drop('DPCapacityInMaintenance',axis=1).drop('DPCheckSum',axis=1)

In [24]:
bb.head()

,Timestamp,SPCapacityAvailable
3671,2018-01-24 22:45:41,43
3672,2018-01-24 23:00:02,43
3673,2018-01-24 23:30:02,43
3674,2018-01-25 00:00:02,43
3675,2018-01-25 00:30:05,43


In [25]:
# read weather observations
weather = pd.read_csv('data//weatherhistory.csv')
weather.head()

,id,observationtime,temperature,weather,windspeed,visibility,winddirection,precipation1hr,requestedlocation,observationlocation,updated
0,1,2017-10-22 18:37:22,11,Overcast,2,10,East,0,NaN,NaN,2018-05-31 13:46:32.857000000
1,2,2017-10-22 19:57:24,10,Overcast,2,10,East,0,NaN,NaN,2018-05-31 13:46:32.857000000
2,3,2017-10-22 20:57:57,9,Overcast,2,10,NE,0,NaN,NaN,2018-05-31 13:46:32.857000000
3,4,2017-10-22 21:57:16,9,Overcast,2,10,SE,0,NaN,NaN,2018-05-31 13:46:32.857000000
4,5,2017-10-22 22:57:59,9,Overcast,2,10,NE,0,NaN,NaN,2018-05-31 13:46:32.857000000


In [26]:
# drop columns irrelevant for model
weather = weather.drop('id',axis=1).drop('winddirection',axis=1).drop('precipation1hr',axis=1).drop('requestedlocation',axis=1)\
.drop('observationlocation',axis=1).drop('updated',axis=1)

weather.head(10)

,observationtime,temperature,weather,windspeed,visibility
0,2017-10-22 18:37:22,11,Overcast,2,10
1,2017-10-22 19:57:24,10,Overcast,2,10
2,2017-10-22 20:57:57,9,Overcast,2,10
3,2017-10-22 21:57:16,9,Overcast,2,10
4,2017-10-22 22:57:59,9,Overcast,2,10
5,2017-10-22 23:57:55,8,Mostly Cloudy,2,10
6,2017-10-23 00:38:43,8,Mostly Cloudy,2,10
7,2017-10-23 01:56:57,10,Mostly Cloudy,2,10
8,2017-10-23 02:58:00,10,Mostly Cloudy,2,10
9,2017-10-23 03:56:46,10,Mostly Cloudy,2,10


In [27]:
bb.count()

Timestamp              12769
SPCapacityAvailable    12769
dtype: int64

In [28]:
weather.count()

observationtime    17104
temperature        17104
weather            16107
windspeed          17104
visibility         17104
dtype: int64

In [29]:
# strip off seconds
weather['observationtime'] = weather.observationtime.str[0:16]

weather.head()

,observationtime,temperature,weather,windspeed,visibility
0,2017-10-22 18:37,11,Overcast,2,10
1,2017-10-22 19:57,10,Overcast,2,10
2,2017-10-22 20:57,9,Overcast,2,10
3,2017-10-22 21:57,9,Overcast,2,10
4,2017-10-22 22:57,9,Overcast,2,10


In [30]:
# round to half hours and drop duplicates
import numpy as np
weather['observationtime'] = np.where(weather.observationtime.str[14:16]<'30',\
                                      weather.observationtime.str[0:14]+'00',weather.observationtime.str[0:14]+'30')

weather = weather.drop_duplicates(subset='observationtime', keep='first')

print(weather.head())

bb['Timestamp'] = np.where(bb.Timestamp.str[14:16]<'30',\
                           bb.Timestamp.str[0:14]+'00',bb.Timestamp.str[0:14]+'30')

bb = bb.drop_duplicates(subset='Timestamp', keep='first')

print(bb.head())

    observationtime  temperature   weather  windspeed  visibility
0  2017-10-22 18:30           11  Overcast          2          10
1  2017-10-22 19:30           10  Overcast          2          10
2  2017-10-22 20:30            9  Overcast          2          10
3  2017-10-22 21:30            9  Overcast          2          10
4  2017-10-22 22:30            9  Overcast          2          10
             Timestamp  SPCapacityAvailable
3671  2018-01-24 22:30                   43
3672  2018-01-24 23:00                   43
3673  2018-01-24 23:30                   43
3674  2018-01-25 00:00                   43
3675  2018-01-25 00:30                   43


In [31]:
# merge the to dataframe together based on Timestamp and observationtime
bbweather = pd.merge(bb,weather,left_on='Timestamp',right_on='observationtime')

bbweather.head(10)

,Timestamp,SPCapacityAvailable,observationtime,temperature,weather,windspeed,visibility
0,2018-01-24 22:30,43,2018-01-24 22:30,8,Overcast,1,10
1,2018-01-24 23:00,43,2018-01-24 23:00,8,Rain,3,5
2,2018-01-24 23:30,43,2018-01-24 23:30,8,Overcast,5,13
3,2018-01-25 00:00,43,2018-01-25 00:00,8,Overcast,11,13
4,2018-01-25 01:00,43,2018-01-25 01:00,8,Mostly Cloudy,1,16
5,2018-01-25 01:30,43,2018-01-25 01:30,8,Mostly Cloudy,6,16
6,2018-01-25 02:00,43,2018-01-25 02:00,8,Mostly Cloudy,5,16
7,2018-01-25 02:30,43,2018-01-25 02:30,8,Mostly Cloudy,3,16
8,2018-01-25 03:00,43,2018-01-25 03:00,8,Mostly Cloudy,3,16
9,2018-01-25 03:30,43,2018-01-25 03:30,8,Partly Cloudy,1,16


In [32]:
# extract hour from observationtime and convert to float
bbweather['hour'] = bbweather.observationtime.str[11:13].astype(float)\
+ np.where(bbweather.observationtime.str[14:16] == '00',0,0.5)

In [33]:
bbweather.head(10)

,Timestamp,SPCapacityAvailable,observationtime,temperature,weather,windspeed,visibility,hour
0,2018-01-24 22:30,43,2018-01-24 22:30,8,Overcast,1,10,22.5
1,2018-01-24 23:00,43,2018-01-24 23:00,8,Rain,3,5,23.0
2,2018-01-24 23:30,43,2018-01-24 23:30,8,Overcast,5,13,23.5
3,2018-01-25 00:00,43,2018-01-25 00:00,8,Overcast,11,13,0.0
4,2018-01-25 01:00,43,2018-01-25 01:00,8,Mostly Cloudy,1,16,1.0
5,2018-01-25 01:30,43,2018-01-25 01:30,8,Mostly Cloudy,6,16,1.5
6,2018-01-25 02:00,43,2018-01-25 02:00,8,Mostly Cloudy,5,16,2.0
7,2018-01-25 02:30,43,2018-01-25 02:30,8,Mostly Cloudy,3,16,2.5
8,2018-01-25 03:00,43,2018-01-25 03:00,8,Mostly Cloudy,3,16,3.0
9,2018-01-25 03:30,43,2018-01-25 03:30,8,Partly Cloudy,1,16,3.5


In [34]:
# extract weekday from date
from datetime import date,datetime
bbweather['date'] = bbweather.observationtime.str[0:10]

bbweather['date'] = bbweather['date'].apply(pd.to_datetime,format='%Y-%m-%d')

bbweather['weekday'] = bbweather['date'].apply(date.weekday)

bbweather = bbweather.drop('date',axis=1)

bbweather = bbweather.drop('Timestamp',axis=1)

bbweather = bbweather.drop('observationtime',axis=1)

print(bbweather.head(10))

   SPCapacityAvailable  temperature        weather  windspeed  visibility  \
0                   43            8       Overcast          1          10   
1                   43            8           Rain          3           5   
2                   43            8       Overcast          5          13   
3                   43            8       Overcast         11          13   
4                   43            8  Mostly Cloudy          1          16   
5                   43            8  Mostly Cloudy          6          16   
6                   43            8  Mostly Cloudy          5          16   
7                   43            8  Mostly Cloudy          3          16   
8                   43            8  Mostly Cloudy          3          16   
9                   43            8  Partly Cloudy          1          16   

   hour  weekday  
0  22.5        2  
1  23.0        2  
2  23.5        2  
3   0.0        3  
4   1.0        3  
5   1.5        3  
6   2.0        3  


In [35]:
# use one hot encoding for weather status
bbweather = pd.get_dummies(bbweather, columns=["weather"], prefix=["weather"])
print(bbweather.head(10))

   SPCapacityAvailable  temperature  windspeed  visibility  hour  weekday  \
0                   43            8          1          10  22.5        2   
1                   43            8          3           5  23.0        2   
2                   43            8          5          13  23.5        2   
3                   43            8         11          13   0.0        3   
4                   43            8          1          16   1.0        3   
5                   43            8          6          16   1.5        3   
6                   43            8          5          16   2.0        3   
7                   43            8          3          16   2.5        3   
8                   43            8          3          16   3.0        3   
9                   43            8          1          16   3.5        3   

   weather_Clear  weather_Fog  weather_Ice Pellets  weather_Mostly Cloudy  \
0              0            0                    0                      0  

We will now build a model to predict the number of available blue-bikes (column SPCapacityAvailable) based on hour of the day, day of the week and weather predictions. Building and using the model is very similar to the RandomForestClassifier:

In [36]:
from sklearn.model_selection import train_test_split
X = bbweather.drop('SPCapacityAvailable',axis=1)
y = bbweather['SPCapacityAvailable']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.30)

In [37]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor

# create dictionary of models
modeldict = {'modelForest25':RandomForestRegressor(n_estimators=25),\
             'modelForest50':RandomForestRegressor(n_estimators=50),\
             'modelForest100':RandomForestRegressor(n_estimators=100),\
             'modelForest150':RandomForestRegressor(n_estimators=150),\
             'modelForest200':RandomForestRegressor(n_estimators=200),\
             'modelForest250':RandomForestRegressor(n_estimators=250)}
# initialize MAE by choosing a high value
MAE = 100000
# initialize bestmodel
bestmodel = 'modelForest25'

for modelkey in modeldict:
    model = modeldict[modelkey]

    model.fit(X_train,y_train)

    y_predict = model.predict(X_test)

    NEWMAE = mean_absolute_error(y_test,y_predict)
    if NEWMAE < MAE: 
        MAE = NEWMAE
        bestmodel = modelkey

print('Bestmodel: ' + modelkey)
print('Mean Absolute Error: '+ str(MAE))
r2 = r2_score(y_test,y_predict)
print('R square: ' + str(r2))        

Bestmodel: modelForest250
Mean Absolute Error: 4.754012722412812
R square: 0.6877796659147795


We learn that the more trees the better the prediction (at least till 250 trees). Our predictions from the best model have an error (mean absolute value of deviation) of less than 5 bikes. We could try to further improve the model by e.g. hot encoding the weekday or adding extra features, for example “holiday (y/n)”. 

This model could be used to predict the expected blue-bike usage for the upcoming days based on the weather predictions. The company "Blue-bike" could e.g. provide extra bikes if needed. Remark this is not real time series prediction because we removed the timestamp. This model could lead to discontinuities if you use it to predict the immediate future. 